In [1]:
import sys

sys.path.append('/home/t9s9/PycharmProjects/Multimodal-VAL-Models')

import json
from pathlib import Path

import pandas as pd

from MVALM.datasets import FSD50k

In [108]:
root = Path('/media/t9s9/SSD_ubuntu/datasets/FSD50k/')

df_train = pd.read_csv(root / 'FSD50K.ground_truth/dev.csv')
df_test = pd.read_csv(root / 'FSD50K.ground_truth/eval.csv')
voc = pd.read_csv(root / 'FSD50K.ground_truth/vocabulary.csv',
                  names=['label', 'mids'])

idx_2_label = voc['label'].to_dict()
label_2_idx = {v: k for k, v in idx_2_label.items()}

with open(root / 'FSD50K.metadata/dev_clips_info_FSD50K.json', 'r') as f:
    meta_dev = json.load(f)
with open(root / 'FSD50K.metadata/eval_clips_info_FSD50K.json', 'r') as f:
    meta_test = json.load(f)

voc.to_parquet(root / 'idx_2_label.parquet', index=True)

# Test data

In [34]:
# move files to new location
test_data_dir = root / 'data' / 'test'
test_data_dir.mkdir(exist_ok=True)

for file in (root / 'FSD50K.eval_audio').iterdir():
    file.rename(test_data_dir / file.name)

In [74]:
df_test['labels_str'] = df_test.labels.str.split(',')
df_test['labels'] = df_test['labels_str'].apply(lambda x: list(map(label_2_idx.get, x)))
df_test['filename'] = df_test['fname'].apply(lambda x: str((Path('test') / str(x)).with_suffix('.flac')))

exists = df_test.filename.apply(lambda x: (root / 'data' / x).exists())
assert sum(exists) == len(df_test)

df_test['tags'] = df_test.fname.apply(lambda idx: meta_test[str(idx)]['tags'])
df_test['title'] = df_test.fname.apply(lambda idx: meta_test[str(idx)]['title'])

df_test = df_test.drop(columns=['fname', 'mids'])
df_test.to_parquet(root / 'annot_train.parquet', index=False)

# Train / Val data

In [84]:
# move files to new location
for split in df_train.split.unique():
    data_dir = root / 'data'
    (data_dir / split).mkdir(exist_ok=True)

for i, row in df_train.iterrows():
    file = (root / 'FSD50K.dev_audio' / str(row['fname'])).with_suffix('.flac')
    if file.exists():
        file.rename(root / 'data' / row['split'] / file.name)

In [109]:
df_train['filename'] = df_train.apply(lambda x: str((Path(x['split']) / str(x['fname'])).with_suffix('.flac')), axis=1)
exists = df_train.filename.apply(lambda x: (root / 'data' / x).exists())
assert sum(exists) == len(df_train)

df_train['labels_str'] = df_train.labels.str.split(',')
df_train['labels'] = df_train['labels_str'].apply(lambda x: list(map(label_2_idx.get, x)))

df_train['tags'] = df_train.fname.apply(lambda idx: meta_dev[str(idx)]['tags'])
df_train['title'] = df_train.fname.apply(lambda idx: meta_dev[str(idx)]['title'])

for split, split_df in df_train.groupby('split'):
    split_df = split_df.drop(columns=['fname', 'mids', 'split'])
    split_df.to_parquet(root / f'annot_{split}.parquet', index=False)

# Dataset

In [15]:
root = '/media/t9s9/SSD_ubuntu/datasets/FSD50k/'
dataset = FSD50k(root=root, split='all', sample_rate=16000)
print(dataset)

Dataset: FSD50k, Split: train,val,test, Size: 51197, Label: multilabel


In [22]:
names = []
for i, name in dataset._load_label_names().items():
    name = name.replace('_', ' ').strip()
    name = name[0].lower() + name[1:]
    names.append(name)
names

['accelerating and revving and vroom',
 'accordion',
 'acoustic guitar',
 'aircraft',
 'alarm',
 'animal',
 'applause',
 'bark',
 'bass drum',
 'bass guitar',
 'bathtub (filling or washing)',
 'bell',
 'bicycle',
 'bicycle bell',
 'bird',
 'bird vocalization and bird call and bird song',
 'boat and Water vehicle',
 'boiling',
 'boom',
 'bowed string instrument',
 'brass instrument',
 'breathing',
 'burping and eructation',
 'bus',
 'buzz',
 'camera',
 'car',
 'car passing by',
 'cat',
 'chatter',
 'cheering',
 'chewing and mastication',
 'chicken and rooster',
 'child speech and kid speaking',
 'chime',
 'chink and clink',
 'chirp and tweet',
 'chuckle and chortle',
 'church bell',
 'clapping',
 'clock',
 'coin (dropping)',
 'computer keyboard',
 'conversation',
 'cough',
 'cowbell',
 'crack',
 'crackle',
 'crash cymbal',
 'cricket',
 'crow',
 'crowd',
 'crumpling and crinkling',
 'crushing',
 'crying and sobbing',
 'cupboard open or close',
 'cutlery and silverware',
 'cymbal',
 'dish

In [14]:
dp = dataset[6598]
print(dp.audio.shape)
for l in dp.label:
    print(dataset.idx2label(l))
dp

torch.Size([29895])
Gunshot_and_gunfire
Explosion


AudioOutput(audio=tensor([-2.9692e-01,  2.9281e-01, -1.3140e-01,  ...,  1.6565e-05,
        -2.5099e-06,  8.9923e-06]), audio_fps=16000, label=array([92, 72]), filename='train/391180.flac')